In [22]:
import pandas as pd
observation_board = pd.DataFrame(columns=[
    "Dataset_Size",
    "Num_Particles",        
    "Num_Iterations",      
    "Num_Cities",           
    "Device",               
    "Best_Score",          
    "Time_Taken(s)",        
    "Global_Best_Position"  
])
def log_observation(dataset_size, num_particles, num_iterations, num_cities, device, best_score, time_taken, global_best_position):
    global observation_board
    observation_board = pd.concat([
        observation_board,
        pd.DataFrame([{
            "Dataset_Size": dataset_size,
            "Num_Particles": num_particles,
            "Num_Iterations": num_iterations,
            "Num_Cities": num_cities,
            "Device": device,
            "Best_Score": best_score,
            "Time_Taken(s)": time_taken,
            "Global_Best_Position": global_best_position.cpu().numpy().tolist()
        }])
    ], ignore_index=True)

## With CPU Processing

In [36]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time as time

In [ ]:
dataset_size = "large"
large_data_path = f"/home/nirjhar/Python Codes/Machine Learning/data/archive/{dataset_size}.csv"
large_data = pd.read_csv(large_data_path)
print(large_data)

In [38]:
X, Y = np.array(large_data['X']), np.array(large_data['Y'])

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(X, Y, c=Y, cmap=plt.cm.viridis)
plt.show()

In [ ]:
import numpy as np

# Sample coordinates of cities (replace with your own)
cities = [(X[i], Y[i]) for i in range(len(X))]
# Function to calculate the Euclidean distance
def euclidean_distance(coord1, coord2):
    return np.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# Create the distance matrix
def create_distance_matrix(coords):
    n = len(coords)
    distance_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            distance_matrix[i][j] = euclidean_distance(coords[i], coords[j])
    return distance_matrix

# Generate the distance matrix
distance_matrix = create_distance_matrix(cities)

# Display the result
print("Distance Matrix:")
print(distance_matrix)
print(distance_matrix.shape)

In [ ]:
import random
from tqdm.auto import tqdm

device = 'cpu'
distance_matrix = np.array(distance_matrix)

# Coordinates for cities (for visualization purposes)
city_coordinates = cities

# Function to calculate total distance of a tour
def calculate_total_distance(tour, distance_matrix):
    total_distance = 0
    for i in range(len(tour) - 1):
        total_distance += distance_matrix[tour[i], tour[i + 1]]
    total_distance += distance_matrix[tour[-1], tour[0]]  # Return to start
    return total_distance

# PSO parameters
num_particles = 10
num_iterations = 40000
num_cities = distance_matrix.shape[0]

# Initialize particles to be done by Model in future
particles = [random.sample(range(num_cities), num_cities) for _ in range(num_particles)] # Initialising all the PARTICLES(A Route) in particles=[particle1, particle2, particle3, .... , particleN]
personal_best_positions = particles.copy()
personal_best_scores = [calculate_total_distance(p, distance_matrix) for p in particles] # score is same as distance covered while travelling through the cities
global_best_position = personal_best_positions[np.argmin(personal_best_scores)]
global_best_score = min(personal_best_scores)

best_scores = []

# PSO main loop
start = time.time()

for iteration in tqdm(range(num_iterations)):
    for i in range(num_particles):
        # Velocities defined by swaping random cities in a route for exploration of all routes approximately
        new_position = particles[i][:]
        city1, city2 = random.sample(range(num_cities), 2)
        new_position[city1], new_position[city2] = new_position[city2], new_position[city1]

        new_score = calculate_total_distance(new_position, distance_matrix)

        if new_score < personal_best_scores[i]:
            personal_best_positions[i] = new_position
            personal_best_scores[i] = new_score

        if new_score < global_best_score:
            global_best_position = new_position
            global_best_score = new_score

    particles = personal_best_positions.copy()
    
    # Accumulate the global best score for plotting
    best_scores.append(global_best_score)
    
end = time.time()

log_observation(
        dataset_size=dataset_size,
        num_particles=num_particles,
        num_iterations=num_iterations,
        num_cities=num_cities,
        device=device,
        best_score=global_best_score.item(),
        time_taken=(end - start),
        global_best_position=torch.tensor(global_best_position)
    )

print(f"Best Route : {global_best_position}")
print(f"Best Score = {global_best_score} units")

print(f"Total Time Taken For Optimisation Using {num_particles} Particles , on {num_cities} Cities , for {num_iterations} Iterations = {(end-start)*1000}ms")

# Plot convergence
plt.figure(figsize=(10, 5))
plt.style.use("dark_background")
plt.plot(best_scores, label="Global Best Distance", color="blue")
plt.title("Convergence of PSO for TSP")
plt.xlabel("Iterations")
plt.ylabel("Distance")
plt.legend()
plt.grid()
plt.show()



In [ ]:
# Plot the optimal tour
def plot_tour(tour, coordinates):
    if num_cities == 1000:
        plt.figure(figsize=(100, 100))
    if num_cities == 100:
        plt.figure(figsize=(40, 40))
    if num_cities == 30:
        plt.figure(figsize=(20, 20))
    if num_cities == 10:
        plt.figure(figsize=(10, 10))
    tour_coords = [coordinates[city] for city in tour + [tour[0]]]  # Close the loop
    x, y = zip(*tour_coords)
    plt.style.use("dark_background")
    plt.plot(x, y, marker='o', color='red', label="Optimal Tour")
    for i, coord in enumerate(coordinates):
        plt.text(coord[0], coord[1], f"City {i}", fontsize=12, ha='center', va='center')
    plt.title("Optimal Tour")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.grid()
    plt.show()

plot_tour(global_best_position, city_coordinates)


## With PyTorch GPU Utlisation


In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.auto import tqdm
import pandas as pd

NUM_PARTICLES = 10
NUM_ITERATIONS = 100000
DATASET_SIZE = "large" # large, medium, small, tiny

def euclidean_distance(coords):
    coords = torch.tensor(coords, device=device)
    x_diff = coords[:, 0].unsqueeze(1) - coords[:, 0]
    y_diff = coords[:, 1].unsqueeze(1) - coords[:, 1]
    distance_matrix = torch.sqrt(x_diff**2 + y_diff**2)
    return distance_matrix


def calculate_total_distance_batched(tours, distance_matrix):
    # Computes total distances for all tours in a batch.
    distances = distance_matrix[tours[:, :-1], tours[:, 1:]].sum(dim=1)
    distances += distance_matrix[tours[:, -1], tours[:, 0]]  # Return to start
    return distances


def update_particles_batched(particles, distance_matrix):
    # Updates particles by performing random swaps and evaluating distances.
    new_particles = particles.clone()
    batch_size, num_cities = particles.shape

    # Randomly sample cities to swap
    indices1 = torch.randint(0, num_cities, (batch_size,), device=device)
    indices2 = torch.randint(0, num_cities, (batch_size,), device=device)

    # Swap cities
    new_particles[range(batch_size), indices1], new_particles[range(batch_size), indices2] = (
        new_particles[range(batch_size), indices2],
        new_particles[range(batch_size), indices1],
    )

    # Calculate distances
    new_scores = calculate_total_distance_batched(new_particles, distance_matrix)

    return new_particles, new_scores





# Load data
dataset_size = DATASET_SIZE
large_data_path = f"/home/nirjhar/Python Codes/Machine Learning/data/archive/{dataset_size}.csv"
large_data = pd.read_csv(large_data_path)
X, Y = torch.tensor(large_data['X'].values, device='cuda'), torch.tensor(large_data['Y'].values, device='cuda')

# Create distance matrix
cities = [(X[i], Y[i]) for i in range(len(X))]
device = 'cuda' if torch.cuda.is_available() else 'cpu'
distance_matrix = euclidean_distance(cities)

# PSO parameters
num_particles = NUM_PARTICLES
num_iterations = NUM_ITERATIONS
num_cities = distance_matrix.shape[0]




# Initialize particles
particles = torch.stack([torch.randperm(num_cities, device=device) for _ in range(num_particles)])
personal_best_positions = particles.clone()
personal_best_scores = calculate_total_distance_batched(particles, distance_matrix)
global_best_position = personal_best_positions[torch.argmin(personal_best_scores)]
global_best_score = personal_best_scores.min()
best_scores = []




# PSO main loop
start = time.time()

for iteration in tqdm(range(num_iterations)):
    new_particles, new_scores = update_particles_batched(particles, distance_matrix)

    # Update personal bests
    better_mask = new_scores < personal_best_scores
    personal_best_positions[better_mask] = new_particles[better_mask]
    personal_best_scores[better_mask] = new_scores[better_mask]

    # Update global best
    min_idx = torch.argmin(personal_best_scores)
    if personal_best_scores[min_idx] < global_best_score:
        global_best_score = personal_best_scores[min_idx]
        global_best_position = personal_best_positions[min_idx]

    particles = personal_best_positions.clone()
    best_scores.append(global_best_score.item())

end = time.time()

log_observation(
        dataset_size=dataset_size,
        num_particles=num_particles,
        num_iterations=num_iterations,
        num_cities=num_cities,
        device=device,
        best_score=global_best_score.item(),
        time_taken=(end - start),
        global_best_position=global_best_position
    )


# print(f"Best Route : {global_best_position}")
print(f"Best Score = {global_best_score.item()} units")
print(f"Total Time Taken For Optimization Using {num_particles} Particles, on {num_cities} Cities, for {num_iterations} Iterations = {(end - start):.2f} seconds")





# Plot convergence
plt.figure(figsize=(10, 5))
plt.plot(best_scores, label="Global Best Distance", color="blue")
plt.style.use("dark_background")
plt.title("Convergence of PSO for TSP")
plt.xlabel("Iterations")
plt.ylabel("Distance")
plt.legend()
plt.grid()
plt.show()


# Plot the optimal tour
def plot_tour(tour, coordinates):
    if num_cities == 1000:
        plt.figure(figsize=(100, 100))
    if num_cities == 100:
        plt.figure(figsize=(40, 40))
    if num_cities == 30:
        plt.figure(figsize=(20, 20))
    if num_cities == 10:
        plt.figure(figsize=(10, 10))
    tour_coords = [coordinates[city] for city in tour] + [coordinates[tour[0]]]  # Close the loop
    x, y = zip(*tour_coords)
    plt.style.use("dark_background")
    plt.plot(x, y, marker='o', color='red', label="Optimal Tour")
    for i, coord in enumerate(coordinates):
        plt.text(coord[0], coord[1], f"City {i}", fontsize=8, ha='center', va='center', color='white')
    plt.title("Optimal Tour")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid()
    plt.show()

# Usage of the plot_tour function
city_coordinates = [(X[i].item(), Y[i].item()) for i in range(len(X))]
global_best_position = global_best_position.to('cpu').numpy()  # Convert to CPU and numpy array
plot_tour(global_best_position, city_coordinates)


In [ ]:
observation_board

## Optimised Code For Both CPU And GPU

In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm.auto import tqdm

# Global observation board
observation_board = pd.DataFrame(columns=[
    "Dataset_Size",
    "Num_Particles",
    "Num_Iterations",
    "Num_Cities",
    "Device",
    "Best_Score",
    "Time_Taken(s)",
    "Global_Best_Position"
])

# Logging function
def log_observation(dataset_size, num_particles, num_iterations, num_cities, device, best_score, time_taken, global_best_position):
    global observation_board
    observation_board = pd.concat([
        observation_board,
        pd.DataFrame([{
            "Dataset_Size": dataset_size,
            "Num_Particles": num_particles,
            "Num_Iterations": num_iterations,
            "Num_Cities": num_cities,
            "Device": device,
            "Best_Score": best_score,
            "Time_Taken(s)": time_taken,
            "Global_Best_Position": global_best_position.cpu().numpy().tolist()
        }])
    ], ignore_index=True)

# Euclidean distance computation
def euclidean_distance(coords, device):
    coords = torch.tensor(coords, device=device)
    x_diff = coords[:, 0].unsqueeze(1) - coords[:, 0]
    y_diff = coords[:, 1].unsqueeze(1) - coords[:, 1]
    return torch.sqrt(x_diff**2 + y_diff**2)

# Calculate total distance
def calculate_total_distance_batched(tours, distance_matrix):
    distances = distance_matrix[tours[:, :-1], tours[:, 1:]].sum(dim=1)
    distances += distance_matrix[tours[:, -1], tours[:, 0]]  # Return to start
    return distances

# Update particles with random swaps
def update_particles_batched(particles, distance_matrix):
    new_particles = particles.clone()
    batch_size, num_cities = particles.shape

    indices1 = torch.randint(0, num_cities, (batch_size,), device=distance_matrix.device)
    indices2 = torch.randint(0, num_cities, (batch_size,), device=distance_matrix.device)

    # Perform swaps
    new_particles[range(batch_size), indices1], new_particles[range(batch_size), indices2] = (
        new_particles[range(batch_size), indices2],
        new_particles[range(batch_size), indices1],
    )

    # Calculate distances
    new_scores = calculate_total_distance_batched(new_particles, distance_matrix)
    return new_particles, new_scores

# Main PSO function
def run_pso(dataset_size, num_particles, num_iterations, device):
    # Load data
    data_path = f"/home/nirjhar/Python Codes/Machine Learning/data/archive/{dataset_size}.csv"
    data = pd.read_csv(data_path)
    X, Y = torch.tensor(data['X'].values, device=device), torch.tensor(data['Y'].values, device=device)

    # Distance matrix
    cities = [(X[i], Y[i]) for i in range(len(X))]
    distance_matrix = euclidean_distance(cities, device)

    # PSO initialization (To Be Done By Model In Future)
    num_cities = distance_matrix.shape[0]
    particles = torch.stack([torch.randperm(num_cities, device=device) for _ in range(num_particles)])
    personal_best_positions = particles.clone()
    personal_best_scores = calculate_total_distance_batched(particles, distance_matrix)
    global_best_position = personal_best_positions[torch.argmin(personal_best_scores)]
    global_best_score = personal_best_scores.min()
    best_scores = []

    # Optimization loop
    start = time.time()
    for _ in tqdm(range(num_iterations), desc=f"Particles={num_particles}, Iterations={num_iterations}, Device={device}"):
        new_particles, new_scores = update_particles_batched(particles, distance_matrix)

        # Update personal bests
        better_mask = new_scores < personal_best_scores
        personal_best_positions[better_mask] = new_particles[better_mask]
        personal_best_scores[better_mask] = new_scores[better_mask]

        # Update global best
        min_idx = torch.argmin(personal_best_scores)
        if personal_best_scores[min_idx] < global_best_score:
            global_best_score = personal_best_scores[min_idx]
            global_best_position = personal_best_positions[min_idx]

        particles = personal_best_positions.clone()
        best_scores.append(global_best_score.item())
    end = time.time()

    # Log results
    log_observation(
        dataset_size=dataset_size,
        num_particles=num_particles,
        num_iterations=num_iterations,
        num_cities=num_cities,
        device=device,
        best_score=global_best_score.item(),
        time_taken=(end - start),
        global_best_position=global_best_position
    )

    print(f"Best Score = {global_best_score.item()} units")
    print(f"Time Taken: {(end - start):.2f} seconds using {device.upper()}")

    # Plot convergence
    plt.figure(figsize=(10, 5))
    plt.plot(best_scores, label="Global Best Distance", color="blue")
    plt.title(f"Convergence: Particles={num_particles}, Iterations={num_iterations}, Device={device}")
    plt.xlabel("Iterations")
    plt.ylabel("Distance")
    plt.legend()
    plt.grid()
    plt.show()

    return global_best_position, num_cities, X, Y

# Plot the optimal tour
def plot_tour(tour, coordinates, num_cities):
    if num_cities == 1000:
        plt.figure(figsize=(100, 100))
    if num_cities == 100:
        plt.figure(figsize=(40, 40))
    if num_cities == 30:
        plt.figure(figsize=(20, 20))
    if num_cities == 10:
        plt.figure(figsize=(10, 10))
    tour_coords = [coordinates[city] for city in tour] + [coordinates[tour[0]]]  # Close the loop
    x, y = zip(*tour_coords)
    plt.style.use("dark_background")
    plt.plot(x, y, marker='o', color='red', label="Optimal Tour")
    for i, coord in enumerate(coordinates):
        plt.text(coord[0], coord[1], f"City {i}", fontsize=8, ha='center', va='center', color='white')
    plt.title(f"Optimal Tour: {num_cities} Cities")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid()
    plt.show()

# Experiment configurations
particle_configs = [10, 50, 100]
iteration_configs = [1000, 5000, 10000]
dataset_configss = ['tiny', 'small', 'medium', 'large']
devices = ['cpu', 'cuda'] if torch.cuda.is_available() else ['cpu']

# Run experiments
for DATASET_SIZE in dataset_configss:
    for device in devices:
        for num_particles in particle_configs:
            for num_iterations in iteration_configs:
                best_position, num_cities, X, Y = run_pso(DATASET_SIZE, num_particles, num_iterations, device)
                city_coordinates = [(X[i].item(), Y[i].item()) for i in range(len(X))]
                plot_tour(best_position.cpu().numpy(), city_coordinates, num_cities)

# Display observation board
observation_board
